In [1]:
import omdb
import pandas as pd
import requests
import json
import csv
import time
import os
import dill
import numpy as np

In [2]:
data_path=os.getcwd()+'\\website\\flask_files\\model_db\\'
subs_dir=os.getcwd()+'\\subs_storage\\'
import pickle
def save_obj(obj, name):
    with open(data_path + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, protocol=2)
        
def load_obj(name):
    try:
        with open(data_path + name + '.pkd', 'rb+') as fp:
            return pickle.load(fp)
    except:
        with open(data_path + name + '.pkl', 'rb+') as fp:
            return pickle.load(fp)

In [3]:
with open('credentials.pkd','rb') as fp:
    credentials = dill.load(fp)
api_key = credentials['api_key']

In [4]:
def get_page_args(i):
    params=dict()
    url="https://api.themoviedb.org/3/discover/movie"
    params['api_key'] = api_key
    params['sort_by']='popularity.desc'
    params['page']=i
    params['with_original_language']='en'
    return {"url": url, "params": params}

def get_movdb_ids(response):
    #returns a list of modvb ids from page i
    id_list= [item['id'] for item in response.json()['results']]
    return id_list

def get_page(i):
    #requests info form page i. if ids is empty, which it initially is and the code is OK ids gets filled and returned
    #if ids is empty, which it is and the code is 429 (too many requets), program waits, and doesnt change ids so its still empty
    #so it will run again until it fills it. once filled it wont run and function returns it. ids is a list of movdb ids
    ids=[]
    while ids==[]:
        data=requests.get(**get_page_args(i))
        if data.status_code == 200:
            ids=get_movdb_ids(data)
        elif data.status_code == 429:
            print('Waiting for server for: {}sec. On page {}.'.format(int(data.headers['Retry-After']),i))    
            time.sleep(int(data.headers['Retry-After'])+1)        
    return ids

# Get all the omdb_ids
This part gets all the OMDB_ids from the allowed 1000 pages. Should take about 5 mins.

In [300]:
omdb_ids=[]
for page_nr in range(1,1000):
    omdb_ids=omdb_ids + get_page(page_nr)

Waiting for server for:4 s. On page41.
Waiting for server for:4 s. On page81.
Waiting for server for:4 s. On page121.
Waiting for server for:4 s. On page161.
Waiting for server for:4 s. On page201.
Waiting for server for:4 s. On page241.
Waiting for server for:4 s. On page281.
Waiting for server for:4 s. On page321.
Waiting for server for:4 s. On page361.
Waiting for server for:4 s. On page401.
Waiting for server for:4 s. On page441.
Waiting for server for:4 s. On page481.
Waiting for server for:4 s. On page521.
Waiting for server for:4 s. On page561.
Waiting for server for:4 s. On page601.
Waiting for server for:4 s. On page641.
Waiting for server for:4 s. On page681.
Waiting for server for:4 s. On page721.
Waiting for server for:4 s. On page761.
Waiting for server for:4 s. On page801.
Waiting for server for:4 s. On page841.
Waiting for server for:3 s. On page881.
Waiting for server for:3 s. On page921.
Waiting for server for:3 s. On page961.


dill.dump(omdb_ids, open('omdb_ids.pkd','wb'))

Next time you just run this line to open it.

# Download full list of OMDB_IDS

In [5]:
import gzip
with gzip.open('movie_ids_02_04_2018.json.gz') as f:
    data_test = [json.loads(line) for line in f]

In [6]:
data_test[34]

{'adult': False,
 'id': 164,
 'original_title': "Breakfast at Tiffany's",
 'popularity': 23.8627,
 'video': False}

# Get all the movie_info from omdb_ids

In [7]:
def get_page_args_b(movie_id):
    params=dict()
    url="https://api.themoviedb.org/3/movie/"+str(movie_id)
    params['api_key'] = api_key
    params['append_to_response']= 'keywords'
    return {"url": url, "params": params}

def get_movie_info(omdb_id):
    #requests info form page i. if ids is empty, which it initially is and the code is OK ids gets filled and returned
    #if ids is empty, which it is and the code is 429 (too many requets), program waits, and doesnt change ids so its still empty
    #so it will run again until it fills it. once filled it wont run and function returns it. ids is a list of movdb ids
    data_dict=[]
    while data_dict==[]:
        data=requests.get(**get_page_args_b(omdb_id))
        if data.status_code == 200:
            data_dict=data.json()
        elif data.status_code == 429:
            #print('|Limit reached. Retrying in: {}sec'.format(int(data.headers['Retry-After'])))    
            time.sleep(int(data.headers['Retry-After'])+1)
        else:
            print('Something went wrong. Error code received: {}'.format(data.status_code))
            data_dict=None
    return data_dict

In [9]:
N_total=len(data_test)
print(N_total)

374496


In [ ]:
from tqdm import tqdm
import time
import datetime as datetime

N_total=len(data_test)
t0=time.time()
for start_pt in range(50000,70000,10000):
    movie_DB=[]
    for i in tqdm(range(start_pt,start_pt+10000)):
        #print(i,end=' ')
        try:
            movie_DB.append(get_movie_info(data_test[i]['id']))
        except:
            pass

        if i%200==0:
            #print('\nWriting data to file. {} movies so far. {:.3f} Percent Complete.'.format(i,(i/N_movies)*100))
            save_obj(movie_DB, datetime.datetime.now().strftime('movie_DB_%Y_%b_temp'))
    
    
    #final write
    save_obj(movie_DB, datetime.datetime.now().strftime('movie_DB_%Y_%b_%d_%I_%M_%S_final'))
    #tmp_file_remove
    os.remove(data_path+datetime.datetime.now().strftime('movie_DB_%Y_%b_temp')+".pkl")
    
print('elapsed time:{}'.format(time.time()-t0))

 88%|███████████████████████████████████████████████████████████████████▍         | 8760/10000 [38:34<05:27,  3.78it/s]

In [142]:
movie_database=[]
for item in [item.split(".")[0] for item in os.listdir(data_path) if "movie_DB" in item  and "temp" not in item]:
    movie_database+=load_obj(item)

In [157]:
[item['production_countries'][0] for item in movie_database[0:10]]

[{'iso_3166_1': 'US', 'name': 'United States of America'},
 {'iso_3166_1': 'US', 'name': 'United States of America'},
 {'iso_3166_1': 'US', 'name': 'United States of America'},
 {'iso_3166_1': 'US', 'name': 'United States of America'},
 {'iso_3166_1': 'US', 'name': 'United States of America'},
 {'iso_3166_1': 'US', 'name': 'United States of America'},
 {'iso_3166_1': 'JP', 'name': 'Japan'},
 {'iso_3166_1': 'US', 'name': 'United States of America'},
 {'iso_3166_1': 'JP', 'name': 'Japan'},
 {'iso_3166_1': 'US', 'name': 'United States of America'}]

In [162]:
movie_database[8]['production_countries']

[{'iso_3166_1': 'JP', 'name': 'Japan'},
 {'iso_3166_1': 'US', 'name': 'United States of America'}]

In [163]:
movie_database[45]

{'adult': False,
 'backdrop_path': '/paOiOP7w09bMCXsXnWKeWtYVWdz.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}],
 'homepage': '',
 'id': 79123,
 'imdb_id': 'tt1493815',
 'keywords': {'keywords': [{'id': 1227, 'name': 'cemetery'},
   {'id': 2411, 'name': 'new orleans'},
   {'id': 3358, 'name': 'haunted house'},
   {'id': 10323, 'name': 'psychiatric hospital'},
   {'id': 14536, 'name': 'crime'},
   {'id': 162846, 'name': 'ghost'},
   {'id': 199072, 'name': 'suspended cop'}]},
 'original_language': 'en',
 'original_title': 'Little Murder',
 'overview': 'In post-Katrina New Orleans, a disgraced detective encounters the ghost of a murdered woman who wants to help him identify her killer.',
 'popularity': 1.509904,
 'poster_path': '/vmEfXhOjkEeu17Tw8XNGUAEN4kn.jpg',
 'production_companies': [{'id': 56427,
   'name': 'Mind in Motion Entertainment'},
  {'id': 56428, 'name': 'Little Murder Production Company'},
  {'id